In [1]:
import os
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot

In [2]:
model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
robot = Robot()

In [3]:
# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)
mtx = camera_cal['mtx']
dist = camera_cal['dist']
f_u = mtx[0,0]   # focal center coordinates
f_v = mtx[1,1]
focal_center = np.array([f_u, f_v])

# Mapping between set_motor "speed" and measured wheel angular velocity "omega"
# for 0.1 second motor running time
wheel_calibration = {
    "speed": [0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    "omega": [0.0, 3.85, 9.23, 15.0, 25.8, 29.2, 35.4]
}

# Open Image Widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)

BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)

diag_dir = 'diagnostics'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(diag_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


In [9]:
# Bbox for TV and stool at various camera poses
# For (0,0,0) to (1,0,0)
beg_bbox_TV = [(72, 92), (136, 137)]  # Pose (0,0,0)
mid_bbox_TV = [(33, 69), (114, 124)]  # Pose ~(0.5,0,0)
end_bbox_stool = [(175, 87), (238, 186)]   # Pose (1.0,0,0)

# Bbox for 2nd stool at various camera poses
# For (1,0,0) to (2,0,pi)
mid_bbox_chair = [(159, 69), (240, 184)]  # midpoint to Pose (1,2,0)
end_bbox_plant = [(59, 76), (155, 187)]   # Pose (1,2,0)



In [4]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image (optional)
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def draw_bbox(img, width, height, bbox, color, line_width):
    bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                  (int(width * bbox[2]), int(height * bbox[3]))]
    cv2.rectangle(img, bbox_pixel[0], bbox_pixel[1], color, line_width)
    return bbox_pixel

def display_detected(img, detections, width, height, debug=False):
    """ put blue bounding boxes on detected objects on image """
    
    for det in detections[0]:
        label = COCO_labels[det['label']-1]
        bbox = det['bbox']
        bbox_pixel = draw_bbox(img, width, height, bbox, BLUE, 1)
        if debug:
            print(label,det['label'], bbox_pixel)
    return

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def object_center(obj_features):
    """Computes the center x, y coordinates of the object using its feature points"""
    UL_fp, UR_fp, LR_fp, LL_fp = obj_features
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
    center_x = (UL_fp_x + LR_fp_x) / 2.0 - 0.5
    center_y = (UL_fp_y + LR_fp_y) / 2.0 - 0.5
    return np.array([center_x, center_y])
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections, debug= False):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if debug:
            print(center)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points from a bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL(upper left), UR(upper right), LR(lower right), LL(lower left) feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]

def generate_ref_obj_features(ref_obj, img, detections, width, height, debug=False):
    """ Find reference object in detected objects and generate 4 feature points """
    matching_detections = [d for d in detections[0] if d['label'] == ref_obj]

    # get detection closest to center of field of view and draw it (if model detects multiple
    # reference objects)
    det = closest_detection(matching_detections)

    if det is not None:
        bbox = det['bbox']
        # bound reference object in green
        draw_bbox(img, width, height, bbox, GREEN, 1)
        # convert to pixel units
        bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]
        return create_feature_pts(bbox_pixel) # return 4 feature points of bounding box
    else:
        return None
    
def fp_error(current, goal):
    """ Return error of a feature point """
    current_x, current_y = current
    goal_x, goal_y = goal
    # Use Corke's convention of (p*-p) --> gain is a +ve number
    return (goal_x-current_x, goal_y-current_y)  

def features_error(currentFeatures, goalFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_goal, UR_goal, LR_goal, LL_goal = goalFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_goal)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_goal)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_goal)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_goal)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))

def image_jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0,      u/Z, u*v/f_u,     -(f_u+u*u/f_u), v],
                  [0,      -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v,      -u]])
    return L

def robot2world(robot_orientation):
    """ calculate the robot jacobian """
    theta = robot_orientation
    
    """
    This may be incorrect
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
    """

    return np.array([[0,0],
                  [0,0],
                  [1,0],
                  [0,0],
                  [0,-1],
                  [0,0]])

def control2robot(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    return np.array([[r/2, r/2],
                  [r/l, -r/l]])

def save_snapshot(img, directory, name, i):
    image_path = os.path.join(directory, 'detect_'+name+str(i+1)+'.jpg')
    cv2.imwrite(image_path, img) 
    return

def omega2speed(in_val, mapping):
    """ Map wheel angular speed to motor speed setting based on a calibration mapping """
    
    if in_val < 0:
        sign = -1
        in_val = abs(in_val)
    else:
        sign = 1
        
    out_lower = 0
    in_lower = 0
    out_val = 0

    for i, in_upper in enumerate(mapping["omega"]):
        # print (i, in_upper)
        if in_val < in_upper:
            out_upper = mapping["speed"][i]
            out_val = out_lower + (in_val - in_lower)/(in_upper - in_lower) \
                *(out_upper-out_lower)
            # print("yes", out_val)
            break
        else:
            # print("no")
            out_lower = mapping["speed"][i]
            in_lower = in_upper
            
    if out_val is 0:
        print ("Input is too high!!!", in_val)
        out_val = 0
        
    return sign*out_val

def rotate_left_2wheels(speed, Rtime):
    
    robot.set_motors(-speed, speed)
    time.sleep(Rtime)
    robot.stop()
    
    return

def clamp(wheel_velocities, upper_limit, lower_limit, T, debug=False):
    """ Put limits on motor speed """
    
    robot_velocities = np.dot(T, wheel_velocities)  # get (tranl_vel, ang_vel)
    tranl_velocity = robot_velocities[0,0]
    
    # Set lower limit on translational speed to 0.15m/s (motor setting 0.3)
    if abs(tranl_velocity) < lower_limit:
        robot_velocities[0,0] = tranl_velocity/abs(tranl_velocity) * lower_limit
        if debug:
            print("Robot speed boosted", robot_velocities) 
    # Set upper limit on translational speed to 0.49m/s (motor setting 0.5)
    elif abs(tranl_velocity) > upper_limit:
        robot_velocities[0,0] = tranl_velocity/abs(tranl_velocity) * upper_limit
        if debug:
            print("Robot speed limited", robot_velocities)
            
    ang_velocity = robot_velocities[1,0]
    ang_upper_limit = 0.5
    if abs(ang_velocity) > ang_upper_limit:
        robot_velocities[1,0] = ang_velocity/abs(ang_velocity) * ang_upper_limit
        if debug:
            print("Rotational speed limited", robot_velocities)
            
    # return clamped wheel velocities
    return np.dot(np.linalg.pinv(T), robot_velocities)
        
 

## IBVS Main Procedure

In [5]:
def ibvs_control(robot_params, control_params):
    """ Control loop for IBVS """
    
    # Load control loop parameters
    gain = control_params["gain"]
    Rtime = control_params["Rtime"]
    num_iter = control_params["num_iter"]
    interval = control_params["interval"]
    no_motion = control_params["no_motion"]
    debug = goal_bbox = control_params["debug"]
    ref_obj = control_params["ref_obj"]
    obj_name = control_params["obj_name"]
    goal_bbox = control_params["goal_bbox"]
    error_window = control_params["error_window"]
    speed_lower_limit = control_params["speed_lower"]
    speed_upper_limit = control_params["speed_upper"]
    
    # load robot control parameters
    depth = robot_params["depth"]
    orientation = robot_params["orientation"]
    wheel_radius = robot_params["wheel_radius"]
    axle_length = robot_params["axle_length"]
    
    goal_features = create_feature_pts(goal_bbox)
    print("Tracking", COCO_labels[ref_obj]," to ", goal_features)

    start = time.perf_counter()
    for i in range(num_iter):
        
        print("Iteration ", i)
        
        """ Grab camera image, undistort and detect objects """
        image = undistort(camera.value, mtx, dist) # undistort camera image
        detections = model(image) # Use SSD model to detect objects
        display_detected(image, detections, width, height, debug) # put bounding boxes on detected objects
        
        """ If ref object is detected, bound it in green box """
        obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
    
        if obj_features is None:
            print("Reference object not detected!!!")
            image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image
            continue
        
        """ Generate error vector (goal-current) """
        # Paint ref object's bbox at robot's goal pose in red
        cv2.rectangle(image, goal_features[0], goal_features[2], RED, 2)
        error = features_error(obj_features,goal_features)
        error_norm = np.linalg.norm(error)
    
        if i%interval == 0 and debug:
            save_snapshot(image, diag_dir, obj_name, i)  # save snap shot
            print ("Ref Object:", obj_features)
            print("Error:", error, error_norm)

        if error_norm < error_window:  # exit control loop if error is within acceptable window
            print ("Goal point achieved!!!")
            break
        
        image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image
        
        """ Implementation of IBVS control: w = gain*pseudo_inv(LHT).error """        
        # T transforms wheel angular velocities (R,L) to robot velocities (translation, angular)
        T = control2robot(wheel_radius, axle_length) 
        # H transform robot velocities to world frame velocities (v_x,v_y,v_z, w_x,w_y, w_z)
        H = robot2world(orientation)  
        # Image Jacobian L - formed by stacking image jacobians of 4 fp of the ref object
        UL_fp, UR_fp, LR_fp, LL_fp = obj_features
        L = np.vstack((image_jacobian(UL_fp, mtx, depth), image_jacobian(UR_fp, mtx, depth), \
              image_jacobian(LR_fp, mtx, depth), image_jacobian(LL_fp, mtx, depth)))
        
        # pinv(LHT) Approach 
        # implement w = gain * pinv(LHT).error
        # J = np.dot(L,np.dot(H,T))
        # inv_J = np.linalg.pinv(J)
        # unclamped_velocities = gain * np.dot(np.linalg.pinv(J), error.T)
        
        # Step-by-step pinv() Approach
        inv_L = np.linalg.pinv(L)
        world_velocities = np.dot(inv_L, error.T)

        inv_H = np.linalg.pinv(H)
        robot_velocities = np.dot(inv_H, world_velocities)

        inv_T = np.linalg.pinv(T)
        unclamped_velocities = np.dot(inv_T, robot_velocities)

        """ Clamp motor setting to (0.3-0.5) """
        wheel_velocities = clamp(unclamped_velocities, speed_upper_limit, \
                                 speed_lower_limit, T, debug) 
        robot_velocities = np.dot(T, wheel_velocities)
        
        
        """ Map wheel angular velocities to motor setting, then run motors """ 
        w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
        w_l = omega2speed(wheel_velocities[1,0],wheel_calibration)
        
        if no_motion is False:
            robot.set_motors(w_l, w_r)  # left, right
            time.sleep(Rtime)
            robot.stop()
       
        """ Update robot's depth and orientation """ 
        transl_velocity = robot_velocities[0,0]
        ang_velocity = robot_velocities[1,0]
        # depth = online_est()  TBD
        
        if i%interval == 0 and debug:
            print("Unclamped robot Velocities:", unclamped_velocities)
            print("Clamped wheel Velocities:", wheel_velocities, (w_l, w_r))
            print("Clamped robot Velocities:", robot_velocities)
            print("Depth:", depth)
            
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image        
    save_snapshot(image, diag_dir, obj_name, i)    # save the last image
    end = time.perf_counter()
    print ("FPS: {:.1f}".format(i/(end-start)))
    
    return

### (0,0,0) --> (1,0,0)

In [30]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

robot_params = {
    "depth": 2.0,
    "orientation": 0,
    "wheel_radius": 0.0325,
    "axle_length": 0.12
}

control_params = {
    "gain": 0.03,
    "Rtime": 0.1, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 3,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 72,  # Use TV as reference object
    "obj_name": "TV",
    "goal_bbox": mid_bbox_TV,  # This should get the robot to ~(0.5,0,0) 
    "error_window": 15,
    "speed_lower": 0.14,  # This ensures motor settings > ~0.3
    "speed_upper": 0.25  # This ensures motor settings > ~0.5
}

ibvs_control(robot_params, control_params)
robot.stop()

Tracking laptop  to  [(33, 69), (114, 69), (114, 124), (33, 124)]
Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19
Iteration  20
Iteration  21
Iteration  22
Iteration  23
Iteration  24
Iteration  25
Iteration  26
Iteration  27
Iteration  28
Iteration  29
Iteration  30
Iteration  31
Iteration  32
Goal point achieved!!!
FPS: 3.5


In [31]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

robot_params = {
    "depth": 1.5,
    "orientation": 0,
    "wheel_radius": 0.0325,
    "axle_length": 0.12
}

control_params = {
    "gain": 0.03,
    "Rtime": 0.1, # The loop run at 8fps --> 0.125s duration
    "num_iter": 50,
    "interval": 2,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 62,  # Use TV as reference object
    "obj_name": "stool",
    "goal_bbox": end_bbox_stool,  # This should get the robot to ~(1,0,0) 
    "error_window": 12,
    "speed_lower": 0.14,  # This ensures motor settings > ~0.3
    "speed_upper": 0.20  # This ensures motor settings > ~0.5
}

ibvs_control(robot_params, control_params)
robot.stop()

Tracking couch  to  [(175, 87), (238, 87), (238, 186), (175, 186)]
Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19
Iteration  20
Iteration  21
Goal point achieved!!!
FPS: 3.5


## (1,0,0) --> (1,2,pi)

In [6]:
def rotate_n_detect(control_param):
    """ The robot rotates left in place until a ref object is close to its focal center """

    # Load control loop parameters
    Rtime_course = control_params["Rtime_course"]
    Rtime_fine = control_params["Rtime_fine"]
    num_iter = control_params["num_iter"]
    interval = control_params["interval"]
    debug = control_params["debug"]
    ref_obj = control_params["ref_obj"]
    obj_name = control_params["obj_name"]
    error_window = control_params["error_window"]
    speed = control_params["speed"]    

    print("Looking for ", COCO_labels[ref_obj], "(", ref_obj, ")")
    Rtime = Rtime_course  # rotate is bigger steps

    for i in range(num_iter):
        
        rotate_left_2wheels(speed, Rtime)  # rotate left by 1 step
        
        """ Identify ref object amongst detected objects """
        image = undistort(camera.value, mtx, dist) # undistort camera image
        detections = model(image) # Use SSD model to detect objects
        display_detected(image, detections, width, height, debug) # put bounding boxes on detected objects
    
        # Detect reference object in camera image and place green bounding box around it
        obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
        
        if i%interval == 0 and debug:
            print(COCO_labels[ref_obj], " identified at:", obj_features)

        # update image widget with camera image
        image_widget.value = bgr8_to_jpeg(image)  
        
        """ Stop rotating if ref object is close to focal center """
        if obj_features is None:
            continue
        else:
            Rtime = Rtime_fine # rotate in finer steps
            
            # Calculate distance of ref obj from focal center
            norm_distance = np.linalg.norm(focal_center - object_center(obj_features))
            print(COCO_labels[ref_obj], " identified! Distance from focal center:", norm_distance)
            if  norm_distance < error_window:
                print("Reference object near focal center !!!")
                break   # exit loop if distance within error window
            continue
    return
    

In [25]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

control_params = {
    "gain": None,
    "Rtime_course": 0.04, # The loop run at 8fps --> 0.125s duration
    "Rtime_fine": 0.02, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 2,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 19,  # Use horse as reference object
    "obj_name": "horse",
    "goal_bbox": None,  # This orients the robot towards a chair, the next ref object
    "error_window": 80,
    "speed": 0.31
}

rotate_n_detect(control_params)
robot.stop()

Looking for  sheep ( 19 )
sheep  identified! Distance from focal center: 100.428282787648
sheep  identified! Distance from focal center: 91.1786880733113
sheep  identified! Distance from focal center: 74.14989166019066
Reference object near focal center !!!


In [26]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

robot_params = {
    "depth": 2.0,
    "orientation": 0,
    "wheel_radius": 0.0325,
    "axle_length": 0.12
}

control_params = {
    "gain": 0.02,
    "Rtime": 0.1, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 5,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 62,  # Use chair as reference object
    "obj_name": "chair",
    "goal_bbox": mid_bbox_chair,  # This should get the robot to ~(0.5,0,0) 
    "error_window": 15,
    "speed_lower": 0.14,  # This ensures motor settings > ~0.3
    "speed_upper": 0.28  # This ensures motor settings > ~0.5
}

ibvs_control(robot_params, control_params)
robot.stop()

Tracking couch  to  [(159, 69), (240, 69), (240, 184), (159, 184)]
Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19
Iteration  20
Iteration  21
Iteration  22
Iteration  23
Iteration  24
Iteration  25
Iteration  26
Iteration  27
Iteration  28
Iteration  29
Iteration  30
Goal point achieved!!!
FPS: 3.5


In [27]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

control_params = {
    "gain": None,
    "Rtime_course": 0.03, # The loop run at 8fps --> 0.125s duration
    "Rtime_fine": 0.015, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 2,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 64,  # Use plant as reference object
    "obj_name": "plant",
    "goal_bbox": None,  # This orients the robot towards a chair, the next ref object
    "error_window": 50,
    "speed": 0.31
}

rotate_n_detect(control_params)
robot.stop()

Looking for  bed ( 64 )
bed  identified! Distance from focal center: 74.00458390995783
bed  identified! Distance from focal center: 62.64900514695552
bed  identified! Distance from focal center: 55.20625902902441
bed  identified! Distance from focal center: 53.33532196617905
bed  identified! Distance from focal center: 46.26608691710758
Reference object near focal center !!!


In [28]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

robot_params = {
    "depth": 2.0,
    "orientation": 0,
    "wheel_radius": 0.0325,
    "axle_length": 0.12
}

control_params = {
    "gain": 0.02,
    "Rtime": 0.1, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 5,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": True,
    "ref_obj": 64,  # Use potted plant as reference object
    "obj_name": "plant",
    "goal_bbox": end_bbox_plant,  # This should get the robot to ~(1,2,0) 
    "error_window": 20,
    "speed_lower": 0.14,  # This ensures motor settings > ~0.3
    "speed_upper": 0.27  # This ensures motor settings > ~0.5
}

ibvs_control(robot_params, control_params)
robot.stop()

Tracking bed  to  [(59, 76), (155, 76), (155, 187), (59, 187)]
Iteration  0
chair 62 [(192, 67), (297, 191)]
potted plant 64 [(93, 113), (146, 176)]
Ref Object: [(93, 113), (146, 113), (146, 176), (93, 176)]
Error: [[-34 -37   9 -37   9  11 -34  11]] 73.85120175054702
Robot speed limited [[ 0.27      ]
 [-0.09731396]]
Unclamped robot Velocities: [[46.85212848]
 [47.21144158]]
Clamped wheel Velocities: [[8.12803576]
 [8.48734886]] (0.3861960754240785, 0.37951739326336936)
Clamped robot Velocities: [[ 0.27      ]
 [-0.09731396]]
Depth: 2.0
Iteration  1
chair 62 [(177, 31), (297, 191)]
potted plant 64 [(91, 120), (146, 178)]
Robot speed limited [[0.27      ]
 [0.35276242]]
Iteration  2
chair 62 [(179, 50), (296, 195)]
potted plant 64 [(88, 117), (145, 178)]
Robot speed limited [[0.27      ]
 [0.26982102]]
Iteration  3
potted plant 64 [(93, 114), (146, 177)]
chair 62 [(177, 6), (297, 190)]
vase 86 [(110, 154), (134, 177)]
Robot speed limited [[ 0.27      ]
 [-0.09825004]]
Iteration  4
Refe

In [152]:
robot.stop()

In [76]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

# Control loop parameters
gain = 1.35
Rtime = 0.1  # The loop run at 8fps --> 0.125s duration
num_iter = 20
interval = 2

no_motion = False  # Flag to disable motor (for debugging)
debug = True

# Use TV as reference object 
ref_obj = 64
goal_features = create_feature_pts(mid_bbox_TV)
print(ref_obj,COCO_labels[ref_obj], goal_features)


start = time.perf_counter()
# Run for fixed # of iterations
for i in range(num_iter):
    
    image = undistort(camera.value, mtx, dist) # undistort camera image
    detections = model(image) # Use SSD model to detect objects
    display_detected(image, detections, width, height, debug=True) # put bounding boxes on detected objects
    
    # Detect reference object in camera image and place green bounding box around it
    obj_features = generate_ref_obj_features(ref_obj, image, detections, width, height)
   
    image_widget.value = bgr8_to_jpeg(image)  # update image widget with camera image

image_widget.value = bgr8_to_jpeg(image)
robot.stop()

end = time.perf_counter()
print ("FPS: {:.1f}".format(num_iter/(end-start)))

64 bed [(33, 69), (114, 69), (114, 124), (33, 124)]
chair 62 [(147, 64), (236, 186)]
potted plant 64 [(80, 116), (126, 177)]
chair 62 [(146, 63), (236, 187)]
potted plant 64 [(81, 117), (127, 177)]
potted plant 64 [(81, 116), (128, 177)]
chair 62 [(145, 64), (236, 187)]
chair 62 [(147, 69), (235, 186)]
potted plant 64 [(80, 116), (127, 177)]
chair 62 [(145, 55), (235, 187)]
potted plant 64 [(81, 116), (127, 177)]
chair 62 [(145, 57), (236, 187)]
potted plant 64 [(81, 117), (127, 177)]
chair 62 [(146, 64), (237, 187)]
potted plant 64 [(81, 116), (127, 177)]
chair 62 [(147, 63), (236, 186)]
potted plant 64 [(81, 116), (127, 177)]
potted plant 64 [(81, 117), (127, 176)]
chair 62 [(149, 71), (235, 185)]
chair 62 [(148, 70), (236, 186)]
potted plant 64 [(82, 116), (127, 177)]
chair 62 [(147, 63), (236, 187)]
potted plant 64 [(81, 115), (127, 177)]
chair 62 [(148, 64), (235, 185)]
potted plant 64 [(80, 114), (127, 177)]
chair 62 [(147, 60), (237, 188)]
potted plant 64 [(81, 114), (127, 176)]

In [29]:
# Display camera image with bounding boxes for detected objects
display(widgets.HBox([image_widget]))

control_params = {
    "gain": None,
    "Rtime_course": 0.04, # The loop run at 8fps --> 0.125s duration
    "Rtime_fine": 0.02, # The loop run at 8fps --> 0.125s duration
    "num_iter": 100,
    "interval": 2,
    "no_motion": False,  # Flag to disable motor (for debugging)
    "debug": False,
    "ref_obj": 74,  # Use horse as reference object
    "obj_name": "mouse",
    "goal_bbox": None,  # This orients the robot towards a chair, the next ref object
    "error_window": 50,
    "speed": 0.31
}

rotate_n_detect(control_params)
robot.stop()

Looking for  remote ( 74 )
remote  identified! Distance from focal center: 19.639619593022797
Reference object near focal center !!!


In [15]:
for i, label in enumerate(COCO_labels):
    print (i+1, label)

1 background
2 person
3 bicycle
4 car
5 airplane
6 bus
7 train
8 truck
9 boat
10 traffic light
11 fire hydrant
12 12
13 stop sign
14 parking meter
15 bench
16 bird
17 cat
18 dog
19 horse
20 sheep
21 cow
22 elephant
23 bear
24 zebra
25 giraffe
26 26
27 backpack
28 umbrella
29 29
30 30
31 handbag
32 tie
33 suitcase
34 frisbee
35 skis
36 snowboard
37 sports ball
38 kite
39 baseball bat
40 baseball glove
41 skateboard
42 surfboard
43 tennis racket
44 bottle
45 45
46 wine glass
47 cup
48 fork
49 knife
50 spoon
51 bowl
52 banana
53 apple
54 sandwich
55 orange
56 broccoli
57 carrot
58 hot dog
59 pizza
60 donut
61 cake
62 chair
63 couch
64 potted plant
65 bed
66 66
67 dining table
68 68
69 69
70 toilet
71 71
72 tv
73 laptop
74 mouse
75 remote
76 keyboard
77 cell phone
78 microwave
79 oven
80 toaster
81 sink
82 refrigerator
83 83
84 book
85 clock
86 vase
87 scissors
88 teddy bear
89 hair drier
90 toothbrush
